# Control FLIR A70 Thermal Camera

## TESTING

This is a testing notebook for controlling a FLIR A70 thermal camera using a python workflow

### Potentially helpful information

IP address of the FLIR camera: 169.254.0.1

### Test 1: connect to the camera

Try using API requests to download the current image from the camera

In [ ]:
# Packages to load
import requests
import os
from datetime import datetime, date
import time

# Frequency (seconds) between camera captures
capture_freq_seconds = 5

# Total length of capture session
capture_length_seconds = 60

# Directory to store images
output_dir = os.path.join(os.getcwd(), "raw_rjpeg_images")
if not os.path.exists(output_dir):
    os.makedirs(output_dir) 

# Set the IP address of the FLIR camera
flirip = "169.254.0.1" # IP address of the camera


In [2]:
# Start time
start_time = time.time()
current_time = time.time()

# Get the date
clocaltime = time.localtime()

# API request for a IR image
api_request_ir = "http://" + flirip + "/api/image/current?imgformat=JPEG"
# API request for a visual image
api_request_visual = "http://" + flirip + "/api/image/current?imgformat=JPEG_visual_small"

# Capture a RGB image at the beginning
filename = output_dir + "/hotbox_flir_visual_" + time.strftime("%Y%m%d_%H%M%S", clocaltime) + ".jpg"
with open(filename, 'wb') as f:
    f.write(requests.get(api_request_visual).content)

# Loop and save a bunch of images
i = 0
while current_time - start_time <= capture_length_seconds:
    
    # Reset the current time
    current_time = time.time()
    
    # Get date and time
    clocaltime = time.localtime()

    filename = output_dir + "/hotbox_flir_rjpeg_frame_" + str(i).zfill(3) + "_" + time.strftime("%Y%m%d_%H%M%S", clocaltime) + ".jpg"
    with open(filename, 'wb') as f:
        f.write(requests.get(api_request_ir).content)
    
    
    # Delay for capture_freq_seconds
    time.sleep(capture_freq_seconds)
    
    # Advance frame count
    i+=1


In [4]:
current_time - start_time

60.67131853103638

In [ ]:
import flirimageextractor
from matplotlib import cm

flir = flirimageextractor.FlirImageExtractor(palettes=[cm.jet, cm.bwr, cm.gist_ncar])

# Read in an image
images = os.listdir(output_dir)

img = images[0]
